# Import Library

In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.decomposition import TruncatedSVD, NMF
from sklearn.preprocessing import MinMaxScaler

# Get Data

In [2]:
def load_data():
    user_game_df = pd.read_csv("../raw_data/rawg_user_games.csv")
    user_game_df['purchase'] = 1
    return user_game_df

In [3]:
data = load_data()
data

,Unnamed: 0,user_id,game_id,game_slug,game_name,user_rating,metacritic,rawg_rating,released,play_time,purchase
0,0,1,446900,high-rise-a-puzzle-cityscape,High Rise - A Puzzle Cityscape,4,NaN,0.00,2020-05-16,0,1
1,1,1,264830,twinfold,Twinfold,5,NaN,0.00,2018-12-05,0,1
2,2,1,58443,pocket-run-pool,Pocket-Run Pool,4,NaN,0.00,2018-05-24,0,1
3,3,1,51288,cinco-paus,Cinco Paus,0,NaN,0.00,2017-12-25,0,1
4,4,1,21889,desert-fox-the-battle-of-el-alamein,Desert Fox: The Battle of El Alamein,4,NaN,0.00,2014-06-26,0,1
...,...,...,...,...,...,...,...,...,...,...,...
255361,255361,999,4062,bioshock-infinite,BioShock Infinite,0,94.0,4.39,2013-03-26,12,1
255362,255362,999,802,borderlands-2,Borderlands 2,0,89.0,4.05,2012-09-18,10,1
255363,255363,999,5679,the-elder-scrolls-v-skyrim,The Elder Scrolls V: Skyrim,0,94.0,4.42,2011-11-11,45,1
255364,255364,999,3328,the-witcher-3-wild-hunt,The Witcher 3: Wild Hunt,0,92.0,4.67,2015-05-18,50,1


In [4]:
data[['user_rating']].value_counts()

user_rating
0              230988
4                9723
3                6435
5                5798
1                2422
dtype: int64

# Rating filtering Matrix

In [5]:
R = data[['user_id', 'game_name', 'purchase']]
R

,user_id,game_name,purchase
0,1,High Rise - A Puzzle Cityscape,1
1,1,Twinfold,1
2,1,Pocket-Run Pool,1
3,1,Cinco Paus,1
4,1,Desert Fox: The Battle of El Alamein,1
...,...,...,...
255361,999,BioShock Infinite,1
255362,999,Borderlands 2,1
255363,999,The Elder Scrolls V: Skyrim,1
255364,999,The Witcher 3: Wild Hunt,1


In [31]:
R_train, R_test = train_test_split(R, test_size=0.01, random_state=37)

In [32]:
R_train

,user_id,game_name,purchase
5613,29,Too Human,1
251908,983,Endless Space,1
116195,450,PAYDAY 2,1
18624,105,Batman: Arkham City - Game of the Year Edition,1
74432,283,Tomb Raider IV: The Last Revelation,1
...,...,...,...
197596,722,Anoxemia,1
162043,590,Zen Pinball 2,1
205644,750,Alpha Prime,1
20843,112,Pid,1


In [33]:
R_test

,user_id,game_name,purchase
131748,507,Medusa's Labyrinth,1
215720,807,Age of Empires II HD,1
40266,176,Salt and Sanctuary,1
71223,277,Need For Speed: Hot Pursuit,1
189604,685,LEGO Batman,1
...,...,...,...
218670,821,Vampire: The Masquerade - Bloodlines,1
110275,434,Nethergate: Resurrection,1
77907,295,Hostile Waters: Antaeus Rising,1
227100,862,Gothic 3,1


In [68]:
def transform_df(df, val):
    game_matrix_df = df.pivot(index = 'game_name', columns ='user_id', values = val).fillna(0)
    return game_matrix_df


In [69]:
R_matrix = transform_df(R_train, 'purchase')
R_matrix

user_id,1,2,3,4,5,6,7,8,9,11,...,986,988,989,991,993,994,995,997,998,999
game_name,,,,,,,,,,,,,,,,,,,,,
! That Bastard Is Trying To Steal Our Gold !,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
!AnyWay!,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
!Dead Pixels Adventure!,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"""BUTTS: The VR Experience""",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"""Glow Ball"" - The billiard puzzle game",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
龙魂时刻,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
신기록: A Compendium of Ghosts,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Ｄｒｅａｍ Ｄｅａｌｅｒ △,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [70]:
def svd_inverse(df, nb_components):
    svd = TruncatedSVD(n_components=nb_components)
    games_factors = svd.fit_transform(df) 
    r = svd.inverse_transform(games_factors)
    return pd.DataFrame(r, index=df.index, columns=df.columns), 

In [92]:
R_pred_svd_inv = svd_inverse(R_matrix, 200)
R_pred_svd_inv

user_id,1,2,3,4,5,6,7,8,9,11,...,986,988,989,991,993,994,995,997,998,999
game_name,,,,,,,,,,,,,,,,,,,,,
! That Bastard Is Trying To Steal Our Gold !,-0.004018,-0.006319,-0.005308,-0.001256,0.000014,-0.000211,-0.000346,-0.003519,-0.000749,-0.000422,...,0.000912,-0.003468,-0.003540,0.000477,-0.003846,-0.003677,0.000141,-0.015867,-0.003357,0.001163
!AnyWay!,0.004423,-0.001754,-0.001476,0.000405,-0.000276,0.000450,-0.000150,-0.002561,-0.001359,0.003044,...,0.000354,-0.000968,-0.005439,-0.000972,-0.001464,-0.003817,-0.000052,-0.005217,0.002837,0.006012
!Dead Pixels Adventure!,-0.002986,0.007687,-0.003247,0.000364,0.000811,-0.000622,-0.000111,-0.004358,-0.001463,0.002013,...,0.001057,-0.003371,0.013085,-0.003321,0.003787,-0.009458,-0.000827,0.003078,-0.008418,0.007560
"""BUTTS: The VR Experience""",0.000534,-0.000875,0.000492,-0.000659,0.000223,0.000020,0.000011,-0.000681,0.000425,-0.003329,...,-0.000109,-0.002122,-0.000051,0.000669,-0.003623,0.001762,-0.001109,-0.001145,-0.001112,0.000654
"""Glow Ball"" - The billiard puzzle game",-0.008538,-0.000178,-0.010987,-0.002376,-0.000956,-0.000828,-0.000090,0.015057,-0.004218,-0.002497,...,0.004731,-0.009910,-0.003130,-0.001744,0.003399,0.003273,-0.000693,-0.018986,-0.024679,-0.020035
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
龙魂时刻,-0.001164,-0.002993,0.000505,-0.000136,-0.000090,-0.000287,0.000018,0.000716,-0.000254,-0.000932,...,-0.000160,-0.000454,0.001027,0.000038,-0.001560,0.000879,-0.000414,-0.000697,0.001062,0.003808
신기록: A Compendium of Ghosts,-0.001164,-0.002993,0.000505,-0.000136,-0.000090,-0.000287,0.000018,0.000716,-0.000254,-0.000932,...,-0.000160,-0.000454,0.001027,0.000038,-0.001560,0.000879,-0.000414,-0.000697,0.001062,0.003808
Ｄｒｅａｍ Ｄｅａｌｅｒ △,-0.006421,0.007514,-0.005639,-0.001405,0.000611,-0.000188,-0.000307,-0.009413,-0.002305,0.002892,...,0.002183,-0.009457,0.011765,-0.005321,0.000353,-0.010210,0.001222,-0.015457,-0.004444,0.013110


In [72]:
'Armada: Modern Tanks' in R_pred_svd_inv.index

True

In [73]:
def scale_pred_matrix(df):
    scaler = MinMaxScaler()
    df = pd.DataFrame(scaler.fit_transform(df), index=df.index)
    return df

In [74]:
R_pred_svd_inv_scaled = scale_pred_matrix(R_pred_svd_inv)
R_pred_svd_inv_scaled.shape

(18542, 815)

In [75]:
R_pred_svd_inv_scaled

,0,1,2,3,4,5,6,7,8,9,...,805,806,807,808,809,810,811,812,813,814
game_name,,,,,,,,,,,,,,,,,,,,,
! That Bastard Is Trying To Steal Our Gold !,0.239026,0.223253,0.193825,0.144504,0.241981,0.143049,0.200861,0.266379,0.133412,0.203091,...,0.198131,0.280363,0.168207,0.302644,0.174629,0.222593,0.152540,0.211565,0.211815,0.209340
!AnyWay!,0.243079,0.225988,0.199061,0.149138,0.240446,0.145958,0.200765,0.266112,0.133442,0.206859,...,0.196814,0.281500,0.167547,0.300409,0.175403,0.224139,0.151772,0.214042,0.218351,0.210690
!Dead Pixels Adventure!,0.237053,0.232425,0.194546,0.148667,0.245154,0.140632,0.203740,0.264739,0.132713,0.203101,...,0.196452,0.282698,0.178985,0.292086,0.179672,0.218810,0.149870,0.222769,0.208159,0.208480
"""BUTTS: The VR Experience""",0.238862,0.226635,0.202323,0.145879,0.243152,0.143845,0.202518,0.269430,0.134935,0.201138,...,0.195430,0.280971,0.170159,0.303831,0.173290,0.224552,0.151545,0.218258,0.215459,0.209664
"""Glow Ball"" - The billiard puzzle game",0.242685,0.221588,0.185368,0.141823,0.238312,0.138891,0.204895,0.271288,0.132017,0.202553,...,0.206843,0.280147,0.164612,0.300967,0.174544,0.226227,0.152777,0.207827,0.204788,0.199126
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
龙魂时刻,0.238599,0.226720,0.202087,0.147470,0.241439,0.142590,0.201681,0.269569,0.134308,0.203364,...,0.195117,0.282218,0.170449,0.301888,0.175265,0.224626,0.151148,0.217884,0.216732,0.209489
신기록: A Compendium of Ghosts,0.238599,0.226720,0.202087,0.147470,0.241439,0.142590,0.201681,0.269569,0.134308,0.203364,...,0.195117,0.282218,0.170449,0.301888,0.175265,0.224626,0.151148,0.217884,0.216732,0.209489
Ｄｒｅａｍ Ｄｅａｌｅｒ △,0.238337,0.229328,0.191712,0.144417,0.243910,0.142651,0.203670,0.261825,0.132189,0.203244,...,0.199599,0.279233,0.177535,0.286940,0.178734,0.220003,0.151282,0.216713,0.206671,0.210686


In [76]:
def get_predicted_ratings(matrix, user_id, game_name):
    try:
        pred = matrix[matrix.index == game_name][user_id][0]
    except:
        #print(f"could not find game {game_name} or user {user_id}")
        pred = 0
    return pred

In [77]:
R_pred_svd_inv_scaled[R_pred_svd_inv_scaled.index == '🔴 Circles'][3]

game_name
🔴 Circles    0.156496
Name: 3, dtype: float64

In [78]:
get_predicted_ratings(R_pred_svd_inv_scaled, 205, 'Armada: Modern Tanks')

0.1616986212427056

In [79]:
get_predicted_ratings(R_pred_svd_inv_scaled, 203, 'Armada: Modern Tanks')

0.18049723661337774

In [80]:
def make_y_pred(test_df, matrix):
    test_df_ = test_df.copy()
    y_pred = []
    for _, row in test_df_.iterrows():
        user_id = row['user_id']
        game_name = row['game_name']
        y_pred.append(get_predicted_ratings(matrix, user_id, game_name))
    test_df_['y_pred'] = y_pred
    return test_df_

In [81]:
df_final = make_y_pred(R_test, R_pred_svd_inv_scaled)
df_final

,user_id,game_name,purchase,y_pred
131748,507,Medusa's Labyrinth,1,0.286784
215720,807,Age of Empires II HD,1,0.162004
40266,176,Salt and Sanctuary,1,0.180001
71223,277,Need For Speed: Hot Pursuit,1,0.151207
189604,685,LEGO Batman,1,0.268588
...,...,...,...,...
218670,821,Vampire: The Masquerade - Bloodlines,1,0.000000
110275,434,Nethergate: Resurrection,1,0.166852
77907,295,Hostile Waters: Antaeus Rising,1,0.134480
227100,862,Gothic 3,1,0.000000


In [82]:
#mae

In [83]:
from sklearn.metrics import mean_squared_error, mean_absolute_error

In [84]:
mean_absolute_error(df_final['purchase'], df_final['y_pred'])

0.7848564907599757

## metrics with scoring

In [62]:
ratings_df = data[['user_id', 'game_name', 'user_rating']]
ratings_df

,user_id,game_name,user_rating
0,1,High Rise - A Puzzle Cityscape,4
1,1,Twinfold,5
2,1,Pocket-Run Pool,4
3,1,Cinco Paus,0
4,1,Desert Fox: The Battle of El Alamein,4
...,...,...,...
255361,999,BioShock Infinite,0
255362,999,Borderlands 2,0
255363,999,The Elder Scrolls V: Skyrim,0
255364,999,The Witcher 3: Wild Hunt,0


In [63]:
X_train, X_test = train_test_split(ratings_df, test_size=0.01, random_state=37)

In [65]:
X_train['user_rating'].value_counts()

0    228684
4      9619
3      6374
5      5736
1      2399
Name: user_rating, dtype: int64

In [66]:
X_test.shape

(2554, 3)

In [86]:
ratings_matrix = transform_df(X_train, 'user_rating')
ratings_matrix

user_id,1,2,3,4,5,6,7,8,9,11,...,986,988,989,991,993,994,995,997,998,999
game_name,,,,,,,,,,,,,,,,,,,,,
! That Bastard Is Trying To Steal Our Gold !,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
!AnyWay!,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
!Dead Pixels Adventure!,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"""BUTTS: The VR Experience""",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"""Glow Ball"" - The billiard puzzle game",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
龙魂时刻,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
신기록: A Compendium of Ghosts,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Ｄｒｅａｍ Ｄｅａｌｅｒ △,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [93]:
ratings_matrix[ratings_matrix == 0].value_counts()

1    2    3    4    5    6    7    8    9    11   12   14   15   16   17   18   19   20   21   22   23   24   25   26   27   29   30   31   34   35   36   37   38   39   40   42   43   44   45   46   47   48   49   50   51   52   53   54   55   56   58   60   61   64   66   67   68   69   70   71   72   73   77   78   80   81   82   83   84   85   87   89   90   91   92   94   95   97   98   100  101  102  103  104  105  106  107  108  110  111  112  113  114  117  119  120  121  123  126  128  129  130  131  132  133  134  135  136  137  138  139  141  142  143  144  146  147  148  149  151  153  154  155  157  158  159  160  161  162  163  164  165  166  167  168  169  170  171  173  174  175  176  177  178  179  180  181  182  183  184  185  186  187  188  189  190  191  193  196  197  201  202  203  204  205  206  207  208  209  210  211  213  214  215  216  218  219  220  222  223  224  225  226  227  229  230  231  232  234  236  238  239  240  241  242  243  244  245  246  247  

In [91]:
ratings_pred_svd_inv = svd_inverse(ratings_matrix, 200)
ratings_pred_svd_inv

user_id,1,2,3,4,5,6,7,8,9,11,...,986,988,989,991,993,994,995,997,998,999
game_name,,,,,,,,,,,,,,,,,,,,,
! That Bastard Is Trying To Steal Our Gold !,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
!AnyWay!,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
!Dead Pixels Adventure!,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"""BUTTS: The VR Experience""",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"""Glow Ball"" - The billiard puzzle game",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
龙魂时刻,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
신기록: A Compendium of Ghosts,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Ｄｒｅａｍ Ｄｅａｌｅｒ △,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [94]:
ratings_pred_svd_inv_scaled = scale_pred_matrix(ratings_pred_svd_inv)
ratings_pred_svd_inv_scaled.shape

(18542, 815)

In [95]:
ratings_predictions = make_y_pred(X_test, ratings_pred_svd_inv_scaled)
ratings_predictions

,user_id,game_name,user_rating,y_pred
131748,507,Medusa's Labyrinth,0,0.012652
215720,807,Age of Empires II HD,0,0.000980
40266,176,Salt and Sanctuary,0,0.474325
71223,277,Need For Speed: Hot Pursuit,0,0.000000
189604,685,LEGO Batman,0,0.000000
...,...,...,...,...
218670,821,Vampire: The Masquerade - Bloodlines,0,0.000000
110275,434,Nethergate: Resurrection,0,0.000000
77907,295,Hostile Waters: Antaeus Rising,0,0.000000
227100,862,Gothic 3,0,0.000000


In [97]:
mean_absolute_error(ratings_predictions['user_rating'], ratings_predictions['y_pred'])

0.44243274081760336